In [1]:
from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")

/home/patrick/qdrant_study/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
collection_name = 'vectors4'

In [5]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=4, distance=models.Distance.COSINE))

True

In [6]:
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=1,
            vector=[0.05, 0.61, 0.76, 0.74],
            payload={
                "city": "Berlin",
                "price": 1.99,
            },
        ),
        models.PointStruct(
            id=2,
            vector=[0.19, 0.81, 0.75, 0.11],
            payload={
                "city": ["Berlin", "London"],
                "price": 1.99,
            },
        ),
        models.PointStruct(
            id=3,
            vector=[0.36, 0.55, 0.47, 0.94],
            payload={
                "city": ["Berlin", "Moscow"],
                "price": [1.99, 2.99],
            },
        ),
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
client.query_points(
    collection_name=collection_name,
    query=models.RecommendQuery(
        recommend=models.RecommendInput(
            positive=[1, 2],
            negative=[3, [0.2, 0.3, 0.4, 0.5]],
            strategy=models.RecommendStrategy.AVERAGE_VECTOR,
        )
    ),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(
                    value="London",
                ),
            )
        ]
    ),
    limit=3,
)

QueryResponse(points=[])

In [17]:
filter_ = models.Filter(
    must=[
        models.FieldCondition(
            key="city",
            match=models.MatchValue(
                value="London",
            ),
        )
    ]
)

recommend_queries = [
    models.QueryRequest(
        query=models.RecommendQuery(
            recommend=models.RecommendInput(positive=[1], negative=[2])
        ),
        filter=filter_,
        limit=3,
    ),
    models.QueryRequest(
        query=models.RecommendQuery(
            recommend=models.RecommendInput(positive=[1], negative=[2])
        ),
        filter=filter_,
        limit=3,
    ),
]

client.query_batch_points(
    collection_name=collection_name, requests=recommend_queries
)

[QueryResponse(points=[]), QueryResponse(points=[])]

In [16]:
discover_queries = [
    models.QueryRequest(
        query=models.DiscoverQuery(
            discover=models.DiscoverInput(
                target=[0.2, 0.1, 0.9, 0.7],
                context=[
                    models.ContextPair(
                        positive=1,
                        negative=2,
                    ),
                    models.ContextPair(
                        positive=1,
                        negative=2,
                    ),
                ],
            )
        ),
        limit=10,
    ),
]

client.query_batch_points(
    collection_name=collection_name, requests=discover_queries
)

[QueryResponse(points=[ScoredPoint(id=3, version=0, score=2.7280724, payload=None, vector=None, shard_key=None, order_value=None)])]